# Chat Model、prompt template 与 历史messages管理

[build a chatbot](https://python.langchain.com/docs/tutorials/chatbot/#message-persistence)


**简介：**<br>
本节介绍了：<br>
- 使用 LangGraph 构建一个对话流程，以支持多轮对话。同时可以通过设置 config 中的线程id来随时暂停当前对话、开启新对话或回到之前的对话
- 再次回顾了 ChatPromptTemplate 的用法
- 使用 trim_messages 对历史对话信息进行管理。

In [ ]:
import os
from langchain.chat_models import init_chat_model

os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

if os.environ.get('GOOGLE_API_KEY'):
    print("yes")

model = init_chat_model("gemini-2.5-flash", model_provider="google-genai")


yes


## 1. Chat Model

LangGraph采用图（Graph）结构来表示对话流程，允许开发者定义复杂的非线性流程和条件分支。这种图结构提供了更大的灵活性，使得动态路由和条件分支变得直观和简单。

`State` 是贯穿 LangGraph 整个工作流的核心数据结构，用于存储和传递任务执行过程中的关键信息，它反映了应用在某一时刻的完整快照，例如用户输入、中间结果、工具调用日志等。

In [ ]:
from langchain_core.messages import HumanMessage

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# 定义一个新的图
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": state["messages"] + [response]}  # 保留历史 + 添加新回复

# 定义图中的单个节点
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 增加记忆
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
# 这使我们能够在单个应用程序中支持多个对话线程，
# 这是当您的应用程序有多个用户时的常见要求。
config = {"configurable": {"thread_id": "abc123"}} 
# 可以理解为，是这一次多轮对话的唯一标识符

In [11]:
query = 'hi! I am Lily.'

input_message = [HumanMessage(query)]
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Lily! Nice to meet you.

I'm an AI assistant, and I'm here to help. What can I do for you today?


In [12]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Lily! You told me that in your first message. 😊


In [13]:
print(output)

{'messages': [HumanMessage(content='hi! I am Lily.', additional_kwargs={}, response_metadata={}, id='801e5210-4320-4c0c-b5f5-9b988d1c67c6'), AIMessage(content="Hi Lily! Nice to meet you.\n\nI'm an AI assistant, and I'm here to help. What can I do for you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--aebac02e-d449-47ca-85dc-0c0a91619566-0', usage_metadata={'input_tokens': 7, 'output_tokens': 361, 'total_tokens': 368, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 329}}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='7b24f457-236e-4a78-b058-e9358f9be47e'), AIMessage(content='Your name is Lily! You told me that in your first message. 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP

## 2. Prompt templates

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages([
    ('system', "you talk like a pirate. anwser all questions to th best of your ability."),
    MessagesPlaceholder(variable_name="messages"),
])

In [24]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt_input = prompt_template.invoke({"messages": state["messages"]})
    response = model.invoke(prompt_input)
    return {"messages": state["messages"] + [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [25]:
config = {"configurable": {"thread_id": "abc345"}}
query = "hi! I am Lily."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Lily! A fine name for a lass, it is! Welcome aboard, me heartie! What brings ye to these digital shores, eh?


In [26]:
print(output)

{'messages': [HumanMessage(content='hi! I am Lily.', additional_kwargs={}, response_metadata={}, id='072940e7-9c28-4151-b68a-26db0c03ad27'), AIMessage(content='Ahoy there, Lily! A fine name for a lass, it is! Welcome aboard, me heartie! What brings ye to these digital shores, eh?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--54136557-517f-4281-b90d-2a05aeaf02e9-0', usage_metadata={'input_tokens': 25, 'output_tokens': 73, 'total_tokens': 98, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 40}})]}


#### 如果System信息中也带参数

In [27]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [33]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)

def call_model(state: State):
    prompt = prompt_template.invoke({"messages": state["messages"], "language": state["language"]})
    response = model.invoke(prompt)
    return {'messages': state["messages"] + [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [34]:
config = {"configurable": {"thread_id": 'abc456'}}
query = "hi! i am Lily."
language = 'Chinese'

input_messags = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你好，莉莉！很高兴认识你。有什么我能帮你的吗？


state已固定，可以不必再传递所有参数

In [36]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你的名字是莉莉（Lily）。


In [37]:
print(output)

{'messages': [HumanMessage(content='hi! I am Lily.', additional_kwargs={}, response_metadata={}, id='072940e7-9c28-4151-b68a-26db0c03ad27'), AIMessage(content='你好，莉莉！很高兴认识你。有什么我能帮你的吗？', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--f365524e-8cc0-4d1f-9cd0-0e73c33f6074-0', usage_metadata={'input_tokens': 25, 'output_tokens': 43, 'total_tokens': 68, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 26}}), HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='4a9fae48-bd0c-4e99-91ac-7c54b0b9ceba'), AIMessage(content='你的名字是莉莉（Lily）。', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--705cae7d-3ac7-4271-af8a-33af76d5a1c3-0', usage_metadata=

## 3. 管理对话历史

随着对话次数增加，messages列表会无限增长，溢出LLM的文本窗口。因此，需要对信息进行管理。

In [41]:
from langchain_core.messages import SystemMessage, trim_messages, AIMessage

trimmer = trim_messages(
    max_tokens=35,
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on='human'
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

运用对话管理：
在template之前，即只需要修改
1.  `call_model` 函数
2. `input_messages`变量

In [42]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    print(f"Messages before trimming: {len(state['messages'])}")
    trimmed_messages = trimmer.invoke(state["messages"])
    print(f"Messages after trimming: {len(trimmed_messages)}")
    print("Remaining messages:")
    for msg in trimmed_messages:
        print(f"  {type(msg).__name__}: {msg.content}")
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": state["messages"] + [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [43]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

Messages before trimming: 12
Messages after trimming: 6
Remaining messages:
  SystemMessage: you're a good assistant
  HumanMessage: thanks
  AIMessage: no problem!
  HumanMessage: having fun?
  AIMessage: yes!
  HumanMessage: What is my name?
================================== Ai Message ==================================

I don't know your name! As an AI, I don't have access to your personal information unless you choose to tell me.

What would you like me to call you?


In [44]:
print(output)

{'messages': [SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}, id='80887633-2ac4-40af-ba00-c6898ddcd184'), HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}, id='f9b0111b-518f-455a-986f-2dfae5a396ab'), AIMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='1dde5ceb-b843-4d03-a4b0-a2fb817e3f69'), HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}, id='b39fab3f-2967-4ec0-bd12-451161092523'), AIMessage(content='nice', additional_kwargs={}, response_metadata={}, id='475ea91f-7504-4441-bb22-698219c007bf'), HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}, id='a70c3911-0283-4ba2-85da-982311d4fcb6'), AIMessage(content='4', additional_kwargs={}, response_metadata={}, id='7992448e-e9b2-4fba-9a04-2997fdeb73de'), HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}, id='80509742-3959-4064-abca-e181d6f6d5a7'), AIMessa